# ***Introduction***

Bus lanes are designed to enhance public transport efficiency, allowing buses to move more quickly through congested areas. When cars illegally enter bus lanes, they reduce bus speed, disrupt traffic flow, and contribute to overall congestion. This project aims to address this problem by developing a detection system that can provide real-time monitoring and insights into how these violations affect traffic.


# ***Object detection  Using YOLO With two lines of the Frame***

Object detection is a crucial technique in computer vision for identifying and locating objects within images or videos. Using the YOLOv8 model, which is known for its real-time accuracy and efficiency, we can analyze a video, detect objects that intersect with specific lines in the frame, and save the processed output as an MP4 video.


## ***Step 1: Install the Required Libraries***

We need to install the `ultralytics` library to use YOLOv8 for object detection

In [ ]:
#!pip install ultralytics -q

## ***Step 2: Import the Libraries***
We will import the necessary libraries, including OpenCV for video processing and the YOLO model from the `ultralytics` package.

In [ ]:
import cv2
from ultralytics import YOLO, solutions
import numpy as np


## ***Step 3: Load the YOLO Model***
We will load the pre-trained YOLOv8 model (`yolov8n.pt`) for object detection and tracking.

In [ ]:
model = YOLO('yolov8n.pt')

## ***Step 4: Capture and Process the Video***

We initialized a video capture object with OpenCV to read the video file, verified it opened correctly, and retrieved the video's width and height for proper processing and frame handling.









In [ ]:
cap = cv2.VideoCapture('/content/busses_line.mp4')
assert cap.isOpened(), 'Error reading video file'
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

## ***Step 5: Define the lines of the Frame***
Next, we defined two lines within the video frame by specifying their start and end points. We stored these points in lists to facilitate their use in subsequent analysis.

In [ ]:
line1_start = (345, 0)
line1_end = (195, 500)

line2_start = (300, 0)
line2_end = (490, 600)

line_points1 = [line1_start, line1_end]
line_points2 = [line2_start, line2_end]



'''success, im0 = cap.read()

cv2.line(im0, line1_start, line1_end, (255, 0, 0), 2)  # red line
cv2.line(im0, line2_start, line2_end, (0, 255, 0), 2)  # Green line'''



'success, im0 = cap.read()\n\ncv2.line(im0, line1_start, line1_end, (255, 0, 0), 2)  # red line\ncv2.line(im0, line2_start, line2_end, (0, 255, 0), 2)  # Green line'

## Step 6: Set Up the Video Writer with MP4 Format
We configure the video writer to save the output video in MP4 format, using the appropriate codec.

In [ ]:
video_writer = cv2.VideoWriter('bus_output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

In [ ]:
#background subtractor
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=True)

## ***Step 8: Process the Video Frames***


Next, we defined a function to check if a bounding box intersects with lines. The function calculates the four corner points of the bounding box and uses a helper function to determine if any of these points lie on the specified lines. It returns True if there is an intersection and False otherwise.

In [ ]:
def is_bbox_intersecting_lines(x, y, w, h):


    rect_points = [(x, y), (x + w, y), (x, y + h), (x + w, y + h)]

    def is_point_on_line_segment(px, py, x1, y1, x2, y2):
        if min(x1, x2) <= px <= max(x1, x2) and min(y1, y2) <= py <= max(y1, y2):
            if x1 != x2:
                m = (y2 - y1) / (x2 - x1)
                b = y1 - m * x1
                return abs(py - (m * px + b)) < 1
            else:
                return abs(px - x1) < 1
        return False

    intersects = any(is_point_on_line_segment(px, py, *line1_start, *line1_end) or
                     is_point_on_line_segment(px, py, *line2_start, *line2_end)
                     for (px, py) in rect_points)

    return intersects

This video demonstrates real-time object detection and tracking using a video feed. The system processes each frame by drawing two green lines, applying background subtraction to detect motion, and utilizing the YOLOv8 model to identify and track objects. When cars or trucks intersect with the predefined lines, they are highlighted with bounding boxes and labels. The processed frames are then saved to a video file, showcasing the system's ability to detect and follow objects effectively.

In [ ]:
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print('Video frame is empty or video processing has been successfully completed.')
        break

    # Draw the lines on each frame
    cv2.line(im0, line1_start, line1_end, (0, 255, 0), 2)  # Blue line
    cv2.line(im0, line2_start, line2_end, (0, 255, 0), 2)  # Green line

    # Apply background subtraction for motion detection
    fgmask = fgbg.apply(im0)
    _, thresh = cv2.threshold(fgmask.copy(), 180, 255, cv2.THRESH_BINARY)
    kernel = np.ones((7, 7), np.uint8)
    thresh = cv2.erode(thresh, kernel)
    thresh = cv2.dilate(thresh, None, iterations=6)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Perform object detection using YOLOv8
    results = model(im0)

    for result in results:

        boxes = result.boxes
        for box in boxes:
            bbox = box.xyxy[0].cpu().numpy()
            confidence = box.conf[0].cpu().numpy()
            class_id = int(box.cls[0].cpu().numpy())

            # Check if the detected object is a car or truck
            if class_id == 2 or class_id == 7::
                x1, y1, x2, y2 = map(int, bbox)
                w, h = x2 - x1, y2 - y1

                # Check if the bounding box intersects with any of the lines
                if is_bbox_intersecting_lines(x1, y1, w, h):

                    label = f"ID: {class_id} Conf: {confidence:.2f}"
                    cv2.rectangle(im0, (x1, y1), (x2, y2), (255, 0, 255), 2)
                    cv2.putText(im0, 'CAR DETECTED', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Write the frame to the video
    video_writer.write(im0)




0: 384x640 32 cars, 1 bus, 174.1ms
Speed: 2.5ms preprocess, 174.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 174.1ms
Speed: 1.7ms preprocess, 174.1ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 192.2ms
Speed: 3.9ms preprocess, 192.2ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 270.6ms
Speed: 4.1ms preprocess, 270.6ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 235.2ms
Speed: 2.1ms preprocess, 235.2ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 241.6ms
Speed: 2.4ms preprocess, 241.6ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 234.0ms
Speed: 2.1ms preprocess, 234.0ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 239.6ms
Speed: 2.5ms preprocess, 239.6ms inference, 1.9ms postprocess per image at 

'    # Count objects and draw the middle line\n    im01 = counter1.start_counting(im0, tracks)\n    im02 = counter2.start_counting(im0, tracks)\n    video_writer.write(im01)\n    video_writer.write(im02)'

## ***Step 9: Release Resources***
Finally, we release the video capture and writer objects and close all OpenCV windows.

In [ ]:
cap.release()
video_writer.release()
cv2.destroyAllWindows()

# ***Conclusion***
In this tutorial, we employed the YOLOv8 model to detect objects crossing a designated line in the bus lanes. We processed each frame to track these objects and saved the results to an MP4 file. This approach can be adapted for various object detection applications across different fields. Additionally, enforcement and monitoring can enhance public transit efficiency and alleviate road congestion.